Web Scrapping

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 

In [ ]:
milk_powder  = []
for page in range(1,4):
    url = f'http://www.rxnfinder.org/FADB-China/food/143/={page}'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    milk_powder = soup.find_all('div', class_='panel panel-default')
    

    for milk in milk_powder:
        illegal_additives = milk.find("div",class_="b-list-advert-base__data__price").text
        predicted_additives = milk.find("div",class_="b-list-advert-base__data__title").text
        desc =  milk.find("div",class_="b-list-advert-base__description-text").text
        milk_powder.append({"name":name,"price":price,"desc":desc})
    
df = pd.DataFrame(milk_powder)
df.to_csv("milk_powder.csv")
df.describe()